## Multi-Class Quantum Convolutional Neural Networks

We implement a QCNN trained to classify the digits of the MNIST dataset

In [ ]:
!pip install pennylane
import tensorflow as tf
import pickle
import inspect

from sklearn.metrics import multilabel_confusion_matrix
from sklearn.decomposition import PCA

import pennylane as qml
from pennylane.templates.embeddings import AmplitudeEmbedding, AngleEmbedding
from pennylane import numpy as np
import autograd.numpy as anp

n_qubit = 8 #number of qubit in the circuit
encoding = 'amplitude' #choose the quantum encoding: 'amplitude' or 'angle'
num_classes = 10 # choose how many classes: 4, 6, 8, 10
all_samples = True #True if you want all the samples, False, if you want only 250 samples for each class
seed = 43 #set to None to generate the seed randomly
U_params = 15 #number of parameters of F_2 circuit
num_layer = 1 #number of convolutional layer repetitions
load_params = False #if True load parameters from a file
opt = 'Adam' #choose the optimizer: Adam, QNGO, or GDO
lr = 0.01 #learning rate
epochs = 2 #number of epochs
batch_size = 64 #size of batch


Load the MNIST dataset: split it in train and test, take only 250 samples if all_samples == False.

Take only 256 features if the amplitude encoding is applied, otherwise only 8 if the angle encoding is used.

In [ ]:
"""
It loads the MNIST dataset and then it processes the dataset based on the encoding method, number of classes and if we want all the samples.
param encoding: indicate the quantum encoding used: 'amplitude' or 'angle'
param num_classes: number of classes to be predicted, which samples take from the dataset
param all_samples: True if we want all the samples, False to take only 250 samples for each class
param seed: random_state seed
return X_train, X_test, Y_train, Y_test: the dataset divided in training and test set
"""
def data_load_and_process(encoding, num_classes, all_samples, seed):
	if seed != None:
		tf.random.set_seed(seed)
		np.random.seed(seed)

	(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


	x_train, x_test = x_train[..., np.newaxis] / 255.0, x_test[..., np.newaxis] / 255.0	 # normalize the data

	#check is the user want all the samples
	if not all_samples:
		num_examples_per_class = 250
		selected_indices = []

		# Iterate through each class to select 1000 examples
		for class_label in range(10):
			indices = np.where(y_train == class_label)[0][:num_examples_per_class]
			selected_indices.extend(indices)

		# Filter the training data to contain only the selected examples
		x_train_subset = x_train[selected_indices]
		y_train_subset = y_train[selected_indices]

		# Shuffle the data
		shuffle_indices = np.random.permutation(len(x_train_subset))
		x_train = x_train_subset[shuffle_indices]
		y_train = y_train_subset[shuffle_indices]

	print("Shape of subset training data:", x_train.shape)
	print("Shape of subset training labels:", y_train.shape)

	#take only the number of classes selected
	mask_train = np.isin(y_train, range(0, num_classes))
	mask_test = np.isin(y_test, range(0, num_classes))

	X_train = x_train[mask_train]
	X_test = x_test[mask_test]
	Y_train = y_train[mask_train]
	Y_test = y_test[mask_test]

	print("Shape of subset training data:", X_train.shape)
	print("Shape of subset training labels:", Y_train.shape)

	#check which encoding is used
	#if amplitude encoding is used, then the 256 most important features are taken using PCA
	if encoding == 'amplitude':
		X_train_flat = X_train.reshape(X_train.shape[0], -1)
		X_test_flat = X_test.reshape(X_test.shape[0], -1)
		pca = PCA(n_components = 256)
		X_train = pca.fit_transform(X_train_flat)
		X_test = pca.transform(X_test_flat)
		return X_train, X_test, Y_train, Y_test
	#if amplitude encoding is used, then the 8 most important features are taken using PCA
	elif encoding == 'angle':
		X_train = tf.image.resize(X_train[:], (784, 1)).numpy()
		X_test = tf.image.resize(X_test[:], (784, 1)).numpy()
		X_train, X_test = tf.squeeze(X_train), tf.squeeze(X_test)

		pca = PCA(8)

		X_train = pca.fit_transform(X_train)
		X_test = pca.transform(X_test)

		# Rescale for angle embedding

		X_train, X_test = (X_train - X_train.min()) * (np.pi / (X_train.max() - X_train.min())),\
						  (X_test - X_test.min()) * (np.pi / (X_test.max() - X_test.min()))
		return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = data_load_and_process(encoding, num_classes, all_samples, seed)

Shape of subset training data: (60000, 28, 28, 1)
Shape of subset training labels: (60000,)
Shape of subset training data: (60000, 28, 28, 1)
Shape of subset training labels: (60000,)


Define the QCNN:

1) Create the two circuits which implement the convolutional layer and the circuit which implements the pooling operation

In [ ]:
"""
F_1 circuit of the paper
param params: theta angle of the rotations. parameters to be trained
param wires: qubits to apply the gates
"""
def CC14(params, wires):
	#U_CC14 r = 1
	for i in range(0, len(wires)):
		qml.RY(params[i], wires=wires[i])
	for i in range(0, len(wires)):
		qml.CRX(params[i + len(wires)], wires=[wires[(i - 1) % len(wires)], wires[i]])



	#U_CC14 r = -1 or 3
	for i in range(0, len(wires)):
		qml.RY(params[i + 2 * len(wires)], wires=wires[i])

	if len(wires) % 3 == 0 or len(wires) == 2:
		for i in range(len(wires) - 1, -1, -1):
			qml.CRX(params[i + 3 * len(wires)], wires=[wires[i], wires[(i-1) % len(wires)]])

	else:
		control = len(wires) - 1
		target = (control + 3) % len(wires)
		for i in range(len(wires) - 1, -1, -1):
			qml.CRX(params[i + 3 * len(wires)], wires=[wires[control], wires[target]])

			control = target
			target = (control + 3) % len(wires)

"""
F_2 circuit of the paper
param params: theta angle of the rotations. parameters to be trained
param wires: qubits to apply the gates
"""
def U_SU4(params, wires): # 15 params
	qml.U3(params[0], params[1], params[2], wires=wires[0])
	qml.U3(params[3], params[4], params[5], wires=wires[1])
	qml.CNOT(wires=[wires[0], wires[1]])
	qml.RY(params[6], wires=wires[0])
	qml.RZ(params[7], wires=wires[1])
	qml.CNOT(wires=[wires[1], wires[0]])
	qml.RY(params[8], wires=wires[0])
	qml.CNOT(wires=[wires[0], wires[1]])
	qml.U3(params[9], params[10], params[11], wires=wires[0])
	qml.U3(params[12], params[13], params[14], wires=wires[1])

"""
It implements the pooling circuit
param params: theta angle of the rotations. parameters to be trained
param wires: qubits to apply the gates
"""
def Pooling_ansatz(params, wires): #2 params
	qml.CRZ(params[0], wires=[wires[0], wires[1]])
	qml.PauliX(wires=wires[0])
	qml.CRX(params[1], wires=[wires[0], wires[1]])

2) create the structure of the convolutional layers

In [ ]:
"""
Quantum Circuits for Convolutional layers
param U: unitary that implements the convolution
param params: theta angle of the rotations. parameters to be trained
param U_params: number of parameters which implement a single block of the F_2 circuit
param num_layer: number of repetition of the convolutional layer
param qubits: array that indicate to which qubit apply the convolutional layer
"""
def conv_layer(U, params, U_params, num_layer, qubits):
		param0 = 0
		param1 = len(qubits) * 2

		#add f_1 circuit
		for l in range(num_layer):
			if len(qubits) == 8: #if it is the first layer, the F_1 circuit is "divided"
				for i in range(0, len(qubits), len(qubits)//2):
					U(params[param0: param1], wires = qubits[i: i + len(qubits)//2])
			else:
				param1 += len(qubits) * 2
				U(params[param0: param1], wires = qubits[0: len(qubits)])

			#now add the two-qubit circuit (F_2)
			param0 = param1
			param1 += U_params
			for i in range(0, len(qubits), 2):
				U_SU4(params[param0: param1], wires = [qubits[i % len(qubits)], qubits[(i + 1) % len(qubits)]])

			for i in range(1, len(qubits), 2):
				U_SU4(params[param0: param1], wires = [qubits[i % len(qubits)], qubits[(i + 1) % len(qubits)]])

			param0 = param1
			param1 += len(qubits) * 2

3) create the structure of the pooling layers

In [ ]:
"""
Quantum Circuits for Pooling layers
param V: unitary which implements the pooling operation
param params: theta angle of the rotations. parameters to be trained
"""
def pooling_layer1(V, params):
	V(params, wires=[7, 6])
	V(params, wires=[1, 0])

def pooling_layer2(V, params):
	V(params, wires=[3, 2])
	V(params, wires=[5, 4])

def pooling_layer3(V, params, num_classes):
	if num_classes == 4: #if we need only 4 classes. we trace out another qubit
		V(params, wires=[2,0])
	V(params, wires=[6,4])

4) create the structure of the QCNN

In [ ]:
"""
It implements the structure of the QCNN
param U: unitary F_1
param params: theta angle of the rotations. parameters to be trained
param U_params: number of parameters which implement a single block of the F_2 circuit
param num_classes: how many classes the QNN has to predict
param num_layer: number of repetition of the convolutional layer
"""
def QCNN_structure(U, params, U_params, num_classes, num_layer):
	#divide the number of parameters for each layer: conv layer1, pooling layer 1, conv layer 2, ...
	#U_params indicates the number of parameters of the F_2 circuit (the circuit applied to couple of adjacent qubit)
	#n_qubit * 2: is the number of parameters for the circuit F_1
	param1CL = params[0: (U_params + n_qubit * 2) * num_layer]
	param1PL = params[(U_params + n_qubit * 2) * num_layer: ((U_params + n_qubit * 2) * num_layer) + 2]

	param2CL = params[((U_params + n_qubit * 2) * num_layer) + 2: ((U_params + n_qubit * 2) * num_layer) + 2 + ((U_params + (n_qubit - 2) * 4) * num_layer)]
	param2PL = params[((U_params + n_qubit * 2) * num_layer) + 2 + ((U_params + (n_qubit - 2) * 4) * num_layer):
					  ((U_params + n_qubit * 2) * num_layer) + 2 + ((U_params + (n_qubit - 2) * 4) * num_layer) + 2]

	param3CL = params[((U_params + n_qubit * 2) * num_layer) + 2 + ((U_params + (n_qubit - 2) * 4) * num_layer) + 2:
					  ((U_params + n_qubit * 2) * num_layer) + 2 + ((U_params + (n_qubit - 2) * 4) * num_layer) + 2 + ((U_params + n_qubit * 2) * num_layer)]

	#apply the circuits
	conv_layer(U, param1CL, U_params, num_layer, range(n_qubit))
	pooling_layer1(Pooling_ansatz, param1PL)

	conv_layer(U, param2CL, U_params, num_layer, [0, 2, 3, 4, 5, 6])
	pooling_layer2(Pooling_ansatz, param2PL)

	conv_layer(U, param3CL, U_params, num_layer, [0, 2, 4, 6])

	#if we have only 4, 6 or 8 classes, then we need another pooling layer and we need to trace out:
	#another qubit if we have 6 or 8 classes, because we need only 3 qubits to represent 6 or 8 classes
	#2 qubits if we have 4 classes, because we need only 2 qubits to represent 4 classes/states
	#if we have 10 classes, then we don't apply another pooling layer, because we need 4 qubits
	if num_classes == 4 or num_classes == 6 or num_classes == 8:
		param3PL = params[((U_params + n_qubit * 2) * num_layer) + 2 + ((U_params + (n_qubit - 2) * 4) * num_layer) + 2 + ((U_params + n_qubit * 2) * num_layer):
						 ((U_params + n_qubit * 2) * num_layer) + 2 + ((U_params + (n_qubit - 2) * 4) * num_layer) + 2 + ((U_params + n_qubit * 2) * num_layer) + 2]

		pooling_layer3(Pooling_ansatz, param3PL, num_classes)

Create the QCNN

In [ ]:
"""
define the simulator and the QCNN: encoding + VQC + measurement
param X: sample in input
param params: theta angle of the rotations. parameters to be trained
param U_params: number of parameters which implement a single block of the F_2 circuit
param embedding_type: which encoding is chosen
param num_classes: how many classes the QNN has to predict
param num_layer: number of repetition of the convolutional layer
return result: the probabilities of the states, which are associated to the MNIST classes
"""
dev = qml.device('default.qubit', wires = n_qubit)
@qml.qnode(dev)
def QCNN(X, params, U_params, embedding_type='amplitude', num_classes=10, num_layer = 1):
	# Data Embedding
	if embedding_type == 'amplitude':
		AmplitudeEmbedding(X, wires=range(8), normalize=True)
	elif embedding_type == 'angle':
		AngleEmbedding(X, wires=range(8), rotation='Y')

	# Create the VQC
	QCNN_structure(CC14, params, U_params, num_classes, num_layer)

	#Measures the necessary qubits
	if num_classes == 4:
		result = qml.probs(wires=[0, 4])
	elif num_classes == 6:
		result = qml.probs(wires=[0, 2, 4])
	elif num_classes == 8:
		result = qml.probs(wires=[0, 2, 4])
	else:
		result = qml.probs(wires=[0, 2, 4, 6])

	return result


Training Step:

1) define the loss function

In [ ]:
"""
It computes the cross-entropy loss
param labels: correct classes of the Training set
param predictions: classes predicted by the QCNN
param num_classes: number of classes
return loss: average loss
"""
def cross_entropy(labels, predictions, num_classes):
	epsilon = 1e-15
	num_samples = len(labels)

	num_classes = len(predictions[0])
	Y_true_one_hot = anp.eye(num_classes)[labels]

	loss = 0.0
	for i in range(num_samples):
		predictions[i] = anp.clip(predictions[i], epsilon, 1 - epsilon)
		loss -= anp.sum(Y_true_one_hot[i] * anp.log(predictions[i]))


	return loss / num_samples

In [ ]:
"""
It executes the circuit for each image of the dataset (divided in batches)
param calculate the loss function
param params: the angle to be trained
param X: batches of the training set
param Y: batches of the training set
param U_params: number of parameters which implement a single block of the F_2 circuit
param embedding_type: indicate the chosen encoding )
param circ_layer: number of repetitions of the convolutional layer
return loss: average loss
"""
def cost(params, X, Y, U_params, embedding_type, num_classes, circ_layer):
	predictions = [QCNN(x, params, U_params, embedding_type, num_classes=num_classes, layer = circ_layer) for x in X]


	loss = cross_entropy(Y, predictions, num_classes)

	return loss

2) Execute the training:

In [ ]:
"""
It executes the training of the QNN
param X_train: X training set
param Y_train: Y training set
param U_params: number of parameters which implement a single block of the F_2 circuit
param embedding_type: the encoding method used
param num_classes: number of classes
param num_layer: number of repetitions of conv layer
param loadParams: if True the parameters are loaded from a file (used to continue a stopped training)
param optimizer: the optimizer used
param learning_rate: learning rate of the optimizer
param epochs: number of epochs
param all_samples: it all the samples are used
param batch_size: size of the batches
param seed: if None a random seed is used, otherwise the value in the variable
return params: the trained parameters
"""
def circuit_training(X_train, Y_train, U_params, embedding_type, num_classes, num_layer, loadParams, optimizer, learning_rate, epochs, all_samples, batch_size, seed):
	if seed != None:
		np.random.seed(seed)
		anp.random.seed(seed)

	#calculate the number of parameters
	if num_classes == 10:
		total_params =	((U_params + n_qubit * 2) * num_layer) + 2 + ((U_params + (n_qubit - 2) * 4) * num_layer) + 2 + ((U_params + n_qubit * 2) * num_layer)
	else: #we have to add another pooling layer at the end, so we need two parameters
		total_params =	((U_params + n_qubit * 2) * num_layer) + 2 + ((U_params + (n_qubit - 2) * 4) * num_layer) + 2 + ((U_params + n_qubit * 2) * num_layer) + 2

	#laod the parameters
	if not loadParams:
		params = np.random.randn(total_params, requires_grad=True)
	else:
		fileParams = open('params' + 'L' + str(num_layer) + 'LR' + str(learning_rate) + optimizer + 'C' + str(num_classes) + str(all_samples) + '.obj', 'rb')

		params = pickle.load(fileParams)
		fileParams.close()
		print(params)

	#choose the optimizer
	if optimizer == 'Adam':
		opt = qml.AdamOptimizer(stepsize=learning_rate)
	elif optimizer == 'GDO':
		opt = qml.GradientDescentOptimizer(stepsize=learning_rate)
	else:
		opt = qml.QNGOptimizer(stepsize=learning_rate)

	#start the training
	loss_history = []
	grad_vals = []
	for e in range(0, epochs):
		print("EPOCH: ", e)
		for b in range(0, len(X_train), batch_size):
			if (b + batch_size) <= len(X_train):
				X_batch = [X_train[i] for i in range(b, b + batch_size)]
				Y_batch = [Y_train[i] for i in range(b, b + batch_size)]
			else:
				X_batch = [X_train[i] for i in range(b, len(X_train))]
				Y_batch = [Y_train[i] for i in range(b, len(X_train))]

			if optimizer == 'QNGO':
				metric_fn = lambda p: qml.metric_tensor(QCNN, approx="block-diag")(X_batch, p, U_params, embedding_type, num_classes, num_layer)
				params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, U_params, embedding_type, num_classes, num_layer),
														 	params, metric_tensor_fn=metric_fn)
			else:
				params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, U_params, embedding_type, num_classes, num_layer),
														 	params)


			if b % (batch_size * 100) == 0:
				print("iteration: ", b, " cost: ", cost_new)
				"""
				loss_history.append(cost_new)
				gradient_fn = qml.grad(cost)
				gradients = gradient_fn(params, X_batch, Y_batch, U, U_params, embedding_type, cost_fn, num_classes, num_layer)
				grad_vals.append(gradients[-1])
				print(gradients)
				print("var ", np.var(grad_vals))
				print("mean grad: ", np.mean(grad_vals))
				"""


		#save the novel parameters at the end of each epoch
		fileParams = open('params' + 'L' + str(num_layer) + 'LR' + str(learning_rate) + optimizer + 'C' + str(num_classes) + str(all_samples) + '.obj', 'wb')


		pickle.dump(params, fileParams)
		fileParams.close()
	return params

print("Loss History for circuit with " + encoding)
trained_params = circuit_training(X_train, Y_train, U_params, encoding, num_classes, num_layer, load_params,
								opt, lr, epochs, all_samples, batch_size, seed)

Loss History for circuit with amplitude
EPOCH:  0


c:\Users\Marco\miniconda3\envs\pl_cpu_jup\Lib\site-packages\autograd\numpy\numpy_vjps.py:698: ComplexWarning: Casting complex values to real discards the imaginary part
  onp.add.at(A, idx, x)


iteration:  0  cost:  2.793639592510273
iteration:  6400  cost:  2.3454968749014977
iteration:  12800  cost:  2.350804201469498
iteration:  19200  cost:  2.390058187012758
iteration:  25600  cost:  2.145615944178888
iteration:  32000  cost:  2.1590629945116815
iteration:  38400  cost:  2.1028156791251527
iteration:  44800  cost:  2.269390313856259
iteration:  51200  cost:  2.2079282103008855
iteration:  57600  cost:  2.271460262932687
EPOCH:  1
iteration:  0  cost:  2.198988499641296
iteration:  6400  cost:  2.141810916788035
iteration:  12800  cost:  2.0999173864445906
iteration:  19200  cost:  2.189340618483039
iteration:  25600  cost:  2.0646491568359924
iteration:  32000  cost:  2.132142383087968
iteration:  38400  cost:  2.028629969450609
iteration:  44800  cost:  2.249857211386858
iteration:  51200  cost:  2.1752603866711544
iteration:  57600  cost:  2.0863983708256084


Infere on the test set

In [ ]:
"""
It computes the accuracy on the test set
param predictions: classes predicted
param labels: true classes
param num_classes: number of classes
return accuracy: accuracy
"""
def accuracy_multi(predictions, labels, num_classes):
	correct_predictions = 0


	for l, p in zip(labels, predictions):
		p2 = []
		for i in range(0, num_classes):
			p2.append(p[i])
		predicted_class = np.argmax(p2)	# Find the index of the predicted class with highest probability
		if predicted_class == l:
			correct_predictions += 1

	accuracy = correct_predictions / len(labels)
	return accuracy

"""
It computes the precision, recall, F1 score and Confusion Matrix on the test set
param predictions: classes predicted
param labels: true classes
param num_classes: number of classes
return accuracy: accuracy
"""
def accuracy_test_multiclass(predictions, label, num_classes):
	#confusion matrix

	preds_np = np.array(predictions)
	preds = np.argmax(preds_np[:, :num_classes], axis = 1)

	conf_mat = multilabel_confusion_matrix(label, preds, labels = list(range(num_classes)))
	print(conf_mat)
	precision = []
	recall = []
	f1 = []
	i = 0
	for c in conf_mat:
		precision.append(c[1][1] / (c[1][1] + c[0][1]))
		recall.append(c[1][1] / (c[1][1] + c[1][0]))
		f1.append(2 * (precision[i] * recall[i]) / (precision[i] + recall[i] + np.finfo(float).eps))

		print("precision " + str(i) + ": " + str(precision[i]))
		print("recall " + str(i) + ": " + str(recall[i]))
		print("f1 " + str(i) + ": " + str(f1[i]))
		i += 1

predictions = []

for x in X_test:
	predictions.append(QCNN(x, trained_params, U_params, encoding, num_classes, num_layer))

accuracy = accuracy_multi(predictions, Y_test, num_classes)
print("Accuracy: " + str(accuracy))
accuracy_test_multiclass(predictions, Y_test, num_classes)

Accuracy: 0.5674
[[[8613  407]
  [ 447  533]]

 [[8064  801]
  [  89 1046]]

 [[8465  503]
  [ 479  553]]

 [[8758  232]
  [ 625  385]]

 [[8284  734]
  [ 445  537]]

 [[8886  222]
  [ 659  233]]

 [[8660  382]
  [ 234  724]]

 [[8747  225]
  [ 251  777]]

 [[8736  290]
  [ 490  484]]

 [[8461  530]
  [ 607  402]]]
precision 0: 0.5670212765957446
recall 0: 0.5438775510204081
f1 0: 0.5552083333333332
precision 1: 0.5663237682728749
recall 1: 0.9215859030837005
f1 1: 0.7015425888665324
precision 2: 0.5236742424242424
recall 2: 0.5358527131782945
f1 2: 0.5296934865900381
precision 3: 0.6239870340356564
recall 3: 0.3811881188118812
f1 3: 0.47326367547633663
precision 4: 0.4225019669551534
recall 4: 0.5468431771894093
f1 4: 0.4766977363515312
precision 5: 0.512087912087912
recall 5: 0.26121076233183854
f1 5: 0.345953971789161
precision 6: 0.6546112115732369
recall 6: 0.755741127348643
f1 6: 0.701550387596899
precision 7: 0.7754491017964071
recall 7: 0.7558365758754864
f1 7: 0.76551724137931

# New Section

L# New Section

Learning shallow quantum circuits with local inversions and circuit sewing

1. Visualisation

In [ ]:
!pip install pennylane
import pennylane as qml
import matplotlib.pyplot as plt

# Define a quantum device with 4 qubits
dev = qml.device('default.qubit', wires=4)

@qml.qnode(dev)
def U_test():
    # Apply Hadamard gates to all qubits
    for i in range(4):
        qml.Hadamard(wires=i)

    # Apply CNOT gates
    qml.CNOT(wires=(1, 2))
    qml.CNOT(wires=(2, 3))
    qml.CNOT(wires=(0, 1))

# Draw the circuit
circuit_diagram = qml.draw(U_test)()
print(circuit_diagram)

# Optionally, visualize using matplotlib (not directly supported by draw_mpl)
# Instead, we can use draw() to get a textual representation or use other libraries for graphical representation.
plt.figure(figsize=(8, 4))
plt.text(0.5, 0.5, circuit_diagram, fontsize=12, ha='center', va='center')
plt.axis('off')
plt.show()

2. Local Inversion

In [ ]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt

def U_test():
    for i in range(4):
        qml.Hadamard(i)
    qml.CNOT((1, 2))
    qml.CNOT((2, 3))
    qml.CNOT((0, 1))

qml.draw_mpl(U_test)()
plt.show()

3. Circuit Sewing

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

# Define the quantum device
dev = qml.device("default.qubit", wires=3, shots=1024)

# Define the quantum teleportation circuit
@qml.qnode(dev)
def teleportation_circuit():
    # Create entanglement between qubits 1 and 2
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[1, 2])

    # Prepare an arbitrary state on qubit 0 (e.g., |+>)
    qml.Hadamard(wires=0)

    # Bell state measurement on qubits 0 and 1
    qml.CNOT(wires=[0, 1])
    qml.Hadamard(wires=0)

    # Measure qubits 0 and 1
    m0 = qml.measure(wires=0)
    m1 = qml.measure(wires=1)

    # Conditional operations on qubit 2 using qml.cond
    qml.cond(m0, qml.PauliX)(wires=2)
    qml.cond(m1, qml.PauliZ)(wires=2)

    # Return the measurement of qubit 2
    return qml.sample(wires=2)

# Visualize the circuit before execution
circuit_drawer = qml.draw(teleportation_circuit)
print("Quantum Teleportation Circuit:")
print(circuit_drawer())

# Execute the circuit
results = teleportation_circuit()

# Count the measurement results
counts = np.bincount(results, minlength=2)

# Plot the histogram of results
plt.bar(["0", "1"], counts)
plt.xlabel("Qubit 2 State")
plt.ylabel("Counts")
plt.title("Quantum Teleportation Result on Qubit 2")
plt.show()

# Display the executed circuit with conditional operations applied
print("\nExecuted Quantum Teleportation Circuit with Conditional Operations:")
print(circuit_drawer())

4. Quantum Teleportation

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

# Define the quantum device
dev = qml.device("default.qubit", wires=3, shots=1024)

# Define the quantum teleportation circuit
@qml.qnode(dev)
def teleportation_circuit():
    # Create entanglement between qubits 1 and 2
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[1, 2])

    # Prepare an arbitrary state on qubit 0 (e.g., |+>)
    qml.Hadamard(wires=0)

    # Bell state measurement on qubits 0 and 1
    qml.CNOT(wires=[0, 1])
    qml.Hadamard(wires=0)

    # Measure qubits 0 and 1
    m0 = qml.measure(wires=0)
    m1 = qml.measure(wires=1)

    # Conditional operations on qubit 2 using qml.cond
    qml.cond(m0, qml.PauliX)(wires=2)
    qml.cond(m1, qml.PauliZ)(wires=2)

    # Return the measurement of qubit 2
    return qml.sample(wires=2)

# Visualize the circuit before execution
circuit_drawer = qml.draw(teleportation_circuit)
print("Quantum Teleportation Circuit:")
print(circuit_drawer())

# Execute the circuit
results = teleportation_circuit()

# Count the measurement results
counts = np.bincount(results, minlength=2)

# Plot the histogram of results
plt.bar(["0", "1"], counts)
plt.xlabel("Qubit 2 State")
plt.ylabel("Counts")
plt.title("Quantum Teleportation Result on Qubit 2")
plt.show()

# Display the executed circuit with conditional operations applied
print("\nExecuted Quantum Teleportation Circuit with Conditional Operations:")
print(circuit_drawer())

# New Section

Generalization in QML from few training data

In [ ]:
!pip install pennylane
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import seaborn as sns
import jax;

jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

import optax  # optimization using jax

import pennylane as qml
import pennylane.numpy as pnp

sns.set()

seed = 0
rng = np.random.default_rng(seed=seed)

Breaking down the layers

In [ ]:
def convolutional_layer(weights, wires, skip_first_layer=True):
    """Adds a convolutional layer to a circuit.
    Args:
        weights (np.array): 1D array with 15 weights of the parametrized gates.
        wires (list[int]): Wires where the convolutional layer acts on.
        skip_first_layer (bool): Skips the first two U3 gates of a layer.
    """
    n_wires = len(wires)
    assert n_wires >= 3, "this circuit is too small!"

    for p in [0, 1]:
        for indx, w in enumerate(wires):
            if indx % 2 == p and indx < n_wires - 1:
                if indx % 2 == 0 and not skip_first_layer:
                    qml.U3(*weights[:3], wires=[w])
                    qml.U3(*weights[3:6], wires=[wires[indx + 1]])
                qml.IsingXX(weights[6], wires=[w, wires[indx + 1]])
                qml.IsingYY(weights[7], wires=[w, wires[indx + 1]])
                qml.IsingZZ(weights[8], wires=[w, wires[indx + 1]])
                qml.U3(*weights[9:12], wires=[w])
                qml.U3(*weights[12:], wires=[wires[indx + 1]])

In [ ]:
def pooling_layer(weights, wires):
    """Adds a pooling layer to a circuit.
    Args:
        weights (np.array): Array with the weights of the conditional U3 gate.
        wires (list[int]): List of wires to apply the pooling layer on.
    """
    n_wires = len(wires)
    assert len(wires) >= 2, "this circuit is too small!"

    for indx, w in enumerate(wires):
        if indx % 2 == 1 and indx < n_wires:
            m_outcome = qml.measure(w)
            qml.cond(m_outcome, qml.U3)(*weights, wires=wires[indx - 1])

In [ ]:
def conv_and_pooling(kernel_weights, n_wires, skip_first_layer=True):
    """Apply both the convolutional and pooling layer."""
    convolutional_layer(kernel_weights[:15], n_wires, skip_first_layer=skip_first_layer)
    pooling_layer(kernel_weights[15:], n_wires)


def dense_layer(weights, wires):
    """Apply an arbitrary unitary gate to a specified set of wires."""
    qml.ArbitraryUnitary(weights, wires)


num_wires = 6
device = qml.device("default.qubit", wires=num_wires)


@qml.qnode(device)
def conv_net(weights, last_layer_weights, features):
    """Define the QCNN circuit
    Args:
        weights (np.array): Parameters of the convolution and pool layers.
        last_layer_weights (np.array): Parameters of the last dense layer.
        features (np.array): Input data to be embedded using AmplitudEmbedding."""

    layers = weights.shape[1]
    wires = list(range(num_wires))

    # inputs the state input_state
    qml.AmplitudeEmbedding(features=features, wires=wires, pad_with=0.5)
    qml.Barrier(wires=wires, only_visual=True)

    # adds convolutional and pooling layers
    for j in range(layers):
        conv_and_pooling(weights[:, j], wires, skip_first_layer=(not j == 0))
        wires = wires[::2]
        qml.Barrier(wires=wires, only_visual=True)

    assert last_layer_weights.size == 4 ** (len(wires)) - 1, (
        "The size of the last layer weights vector is incorrect!"
        f" \n Expected {4 ** (len(wires)) - 1}, Given {last_layer_weights.size}"
    )
    dense_layer(last_layer_weights, wires)
    return qml.probs(wires=(0))


fig, ax = qml.draw_mpl(conv_net)(
    np.random.rand(18, 2), np.random.rand(4 ** 2 - 1), np.random.rand(2 ** num_wires)
)
plt.show()

Training the QCNN on the digits dataset

In [ ]:
digits = datasets.load_digits()
images, labels = digits.data, digits.target

images = images[np.where((labels == 0) | (labels == 1))]
labels = labels[np.where((labels == 0) | (labels == 1))]

fig, axes = plt.subplots(nrows=1, ncols=12, figsize=(3, 1))

for i, ax in enumerate(axes.flatten()):
    ax.imshow(images[i].reshape((8, 8)), cmap="gray")
    ax.axis("off")

plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

In [ ]:
def load_digits_data(num_train, num_test, rng):
    """Return training and testing data of digits dataset."""
    digits = datasets.load_digits()
    features, labels = digits.data, digits.target

    # only use first two classes
    features = features[np.where((labels == 0) | (labels == 1))]
    labels = labels[np.where((labels == 0) | (labels == 1))]

    # normalize data
    features = features / np.linalg.norm(features, axis=1).reshape((-1, 1))

    # subsample train and test split
    train_indices = rng.choice(len(labels), num_train, replace=False)
    test_indices = rng.choice(
        np.setdiff1d(range(len(labels)), train_indices), num_test, replace=False
    )

    x_train, y_train = features[train_indices], labels[train_indices]
    x_test, y_test = features[test_indices], labels[test_indices]

    return (
        jnp.asarray(x_train),
        jnp.asarray(y_train),
        jnp.asarray(x_test),
        jnp.asarray(y_test),
    )

In [ ]:
@jax.jit
def compute_out(weights, weights_last, features, labels):
    """Computes the output of the corresponding label in the qcnn"""
    cost = lambda weights, weights_last, feature, label: conv_net(weights, weights_last, feature)[
        label
    ]
    return jax.vmap(cost, in_axes=(None, None, 0, 0), out_axes=0)(
        weights, weights_last, features, labels
    )


def compute_accuracy(weights, weights_last, features, labels):
    """Computes the accuracy over the provided features and labels"""
    out = compute_out(weights, weights_last, features, labels)
    return jnp.sum(out > 0.5) / len(out)


def compute_cost(weights, weights_last, features, labels):
    """Computes the cost over the provided features and labels"""
    out = compute_out(weights, weights_last, features, labels)
    return 1.0 - jnp.sum(out) / len(labels)


def init_weights():
    """Initializes random weights for the QCNN model."""
    weights = pnp.random.normal(loc=0, scale=1, size=(18, 2), requires_grad=True)
    weights_last = pnp.random.normal(loc=0, scale=1, size=4 ** 2 - 1, requires_grad=True)
    return jnp.array(weights), jnp.array(weights_last)


value_and_grad = jax.jit(jax.value_and_grad(compute_cost, argnums=[0, 1]))

In [ ]:
def train_qcnn(n_train, n_test, n_epochs):
    """
    Args:
        n_train  (int): number of training examples
        n_test   (int): number of test examples
        n_epochs (int): number of training epochs
        desc  (string): displayed string during optimization

    Returns:
        dict: n_train,
        steps,
        train_cost_epochs,
        train_acc_epochs,
        test_cost_epochs,
        test_acc_epochs

    """
    # load data
    x_train, y_train, x_test, y_test = load_digits_data(n_train, n_test, rng)

    # init weights and optimizer
    weights, weights_last = init_weights()

    # learning rate decay
    cosine_decay_scheduler = optax.cosine_decay_schedule(0.1, decay_steps=n_epochs, alpha=0.95)
    optimizer = optax.adam(learning_rate=cosine_decay_scheduler)
    opt_state = optimizer.init((weights, weights_last))

    # data containers
    train_cost_epochs, test_cost_epochs, train_acc_epochs, test_acc_epochs = [], [], [], []

    for step in range(n_epochs):
        # Training step with (adam) optimizer
        train_cost, grad_circuit = value_and_grad(weights, weights_last, x_train, y_train)
        updates, opt_state = optimizer.update(grad_circuit, opt_state)
        weights, weights_last = optax.apply_updates((weights, weights_last), updates)

        train_cost_epochs.append(train_cost)

        # compute accuracy on training data
        train_acc = compute_accuracy(weights, weights_last, x_train, y_train)
        train_acc_epochs.append(train_acc)

        # compute accuracy and cost on testing data
        test_out = compute_out(weights, weights_last, x_test, y_test)
        test_acc = jnp.sum(test_out > 0.5) / len(test_out)
        test_acc_epochs.append(test_acc)
        test_cost = 1.0 - jnp.sum(test_out) / len(test_out)
        test_cost_epochs.append(test_cost)

    return dict(
        n_train=[n_train] * n_epochs,
        step=np.arange(1, n_epochs + 1, dtype=int),
        train_cost=train_cost_epochs,
        train_acc=train_acc_epochs,
        test_cost=test_cost_epochs,
        test_acc=test_acc_epochs,
    )

In [ ]:
n_test = 100
n_epochs = 100
n_reps = 100


def run_iterations(n_train):
    results_df = pd.DataFrame(
        columns=["train_acc", "train_cost", "test_acc", "test_cost", "step", "n_train"]
    )

    for _ in range(n_reps):
        results = train_qcnn(n_train=n_train, n_test=n_test, n_epochs=n_epochs)
        results_df = pd.concat(
            [results_df, pd.DataFrame.from_dict(results)], axis=0, ignore_index=True
        )

    return results_df


# run training for multiple sizes
train_sizes = [2, 5, 10, 20, 40, 80]
results_df = run_iterations(n_train=2)
for n_train in train_sizes[1:]:
    results_df = pd.concat([results_df, run_iterations(n_train=n_train)])

In [ ]:
# aggregate dataframe
df_agg = results_df.groupby(["n_train", "step"]).agg(["mean", "std"])
df_agg = df_agg.reset_index()

sns.set_style('whitegrid')
colors = sns.color_palette()
fig, axes = plt.subplots(ncols=3, figsize=(16.5, 5))

generalization_errors = []

# plot losses and accuracies
for i, n_train in enumerate(train_sizes):
    df = df_agg[df_agg.n_train == n_train]

    dfs = [df.train_cost["mean"], df.test_cost["mean"], df.train_acc["mean"], df.test_acc["mean"]]
    lines = ["o-", "x--", "o-", "x--"]
    labels = [fr"$N={n_train}$", None, fr"$N={n_train}$", None]
    axs = [0,0,2,2]

    for k in range(4):
        ax = axes[axs[k]]
        ax.plot(df.step, dfs[k], lines[k], label=labels[k], markevery=10, color=colors[i], alpha=0.8)


    # plot final loss difference
    dif = df[df.step == 100].test_cost["mean"] - df[df.step == 100].train_cost["mean"]
    generalization_errors.append(dif)

# format loss plot
ax = axes[0]
ax.set_title('Train and Test Losses', fontsize=14)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')

# format generalization error plot
ax = axes[1]
ax.plot(train_sizes, generalization_errors, "o-", label=r"$gen(\alpha)$")
ax.set_xscale('log')
ax.set_xticks(train_sizes)
ax.set_xticklabels(train_sizes)
ax.set_title(r'Generalization Error $gen(\alpha) = R(\alpha) - \hat{R}_N(\alpha)$', fontsize=14)
ax.set_xlabel('Training Set Size')

# format loss plot
ax = axes[2]
ax.set_title('Train and Test Accuracies', fontsize=14)
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.set_ylim(0.5, 1.05)

legend_elements = [
    mpl.lines.Line2D([0], [0], label=f'N={n}', color=colors[i]) for i, n in enumerate(train_sizes)
    ] + [
    mpl.lines.Line2D([0], [0], marker='o', ls='-', label='Train', color='Black'),
    mpl.lines.Line2D([0], [0], marker='x', ls='--', label='Test', color='Black')
    ]

axes[0].legend(handles=legend_elements, ncol=3)
axes[2].legend(handles=legend_elements, ncol=3)

axes[1].set_yscale('log', base=2)
plt.show()